In [1]:
import sys
from SentiLib.text import predict_emotions_text
import os
import pyspark
import time
import re       
from pyspark import SparkContext, SparkConf
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt") ## Sentence tokenizer | divide a text
from nltk.stem.snowball import SnowballStemmer
import spacy
!python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()
sb_english = SnowballStemmer('english') ## quitar el plural
englishST = open('/home/user/englishST.txt', 'r', encoding = 'latin-1')
stop = englishST.read().splitlines()

2022-05-17 17:52:31.205996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-05-17 17:52:31.206013: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/fabio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2022-05-17 17:52:33.304288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-05-17 17:52:33.304308: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 5.9 MB/s eta 0:00:00m eta 0:00:010:00:01
You should consider upgrading via the '/home/fabio/anaconda3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
def regex(linea):
    new_text = re.sub("\W+", " ", linea)#elimina puntos comas,etc
    new_text = re.sub("\d+", " ", new_text).strip()#elimina numeros
    return new_text

def lower_case(linea):
    new_text = linea.lower()
    return new_text

def tokenizer(linea):
    new_text = word_tokenize(linea, language="english")
    return new_text

def stop_words(linea):
    new_text = []
    ## remove stop words
    for word in linea:
        if word not in stop:
                new_text.append(word)
    return new_text
    
def stemmer(linea):
    new_text = []
    # Obtenemos el lemma por cada token
    for word in linea:
        new_text.append(sb_english.stem(word))
    return new_text

def lemmatization(linea):
    new_text = []
    # obtenemos los lemmas
    doc = nlp(" ".join(linea))
    for token in doc:
        new_text.append(token.lemma_)
    return new_text

def join(linea):
    new_text = " ".join(linea)
    return new_text

In [3]:
start = time.time()
# Inicialización
conf = SparkConf().setMaster("local[*]").setAppName("mi programa")
sc = SparkContext(conf = conf)
end = time.time()
print (end - start)

2022-05-17 17:52:50,777 WARN util.Utils: Your hostname, fabio resolves to a loopback address: 127.0.1.1; using 192.168.100.23 instead (on interface eno1)
2022-05-17 17:52:50,777 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-17 17:52:51,179 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1.9265198707580566


In [4]:
#Cargamos los datos de entrada
start = time.time()
datosEntrada = sc.textFile("file:///home/user/Datos/texto.txt")
end = time.time()
print (end - start)

0.3407161235809326


In [5]:
start = time.time()
Texto_final = datosEntrada.map(regex).map(lower_case).map(tokenizer).map(stop_words) \
.map(stemmer).map(lemmatization).map(join) #.map(predict_emotions_text)
Texto_final.saveAsTextFile("file:///home/user/Resultados/RDD_Batch")
end = time.time()
print (end - start)

2022-05-17 17:53:58.299125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-05-17 17:53:58.299125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-05-17 17:53:58.299139: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-17 17:53:58.299142: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


4.06328821182251
